# 0. 코드 준비

In [1]:
import lostark as lok

import pandas as pd
import requests
from urllib.request import urlopen
import urllib
import re
from bs4 import BeautifulSoup
import ast # literal_eval을 위해
import json
from pprint import pprint
import time
import datetime

# 경고문 제거
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# #보안에러
# !pip install certifi
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

In [2]:
# !pip install pymysql
# !pip install ipython-sql
# !pip install mysqlclient

In [2]:
### DataBase ###
%reload_ext sql
db, cursor = lok.get_db_cursor()

# 1. Character

Parameters : characterName

Responses : 

    Severname
    Character Name
    Character Level
    Character Class Name
    Item Avg Level
    Item Max Level


In [9]:
# read charaterName_list.txt
with open("./character/pre_characterName_list.txt", "r", encoding="utf-8") as f:
    file_contents = f.read()
    pre_characterName_list = eval(file_contents)
try:
    with open("./character/characterName_list.txt", "r", encoding="utf-8") as f:
        file_contents = f.read()
        characterName_list = eval(file_contents)
except:
    characterName_list = []

cnt = 0
pcl_len = len(pre_characterName_list)

# Collect subcharacter names
for i in range(pcl_len):
    # Calling api
    try:
        character_responses = lok.get_character_responses(pre_characterName_list[i])
    except Exception as e:
        print("get resp error")
        print(f"characterName: {pre_characterName_list[i]}[Error: {e}]")
        with open("./character/error_characterName_list.txt", "a", encoding="utf-8") as f:
            f.write(str(pre_characterName_list[i]) + f": {e}\n")
        continue
        
    # remove None data
    if character_responses == None
        with open("./character/removed_characterName_list.txt", "a", encoding="utf-8") as f:
            f.write(str(pre_characterName_list[i]) + ": character_responses is None\n")
        pre_characterName_list.remove(pre_characterName_list[i])
        with open("./character/pre_characterName_list.txt", "w", encoding="utf-8") as f:
            f.write(str(pre_characterName_list))   
        continue
        
    try:
        for c in character_responses:
            if (c['CharacterName'] not in characterName_list):
                characterName_list.append(c['CharacterName'])
    except Exception as e:
        print("characterName append error")
        print(f"characterName: {pre_characterName_list[i]}[", f"Error: {e}]")
        break
        
    if i % 2000 == 0:
        print(c['CharacterName'],": ", i)
        print("Current time:", datetime.datetime.now())
        # 중간중간 저장
        with open("./character/characterName_list.txt", "w", encoding="utf-8") as f:
            f.write(str(characterName_list))

get resp error


IndexError: list index out of range

In [19]:
characterName_list += pre_characterName_list
characterName_list = list(set(characterName_list))
with open("./character/characterName_list.txt", "w", encoding="utf-8") as f:
    f.write(str(characterName_list))

In [20]:
with open("./character/characterName_list.txt", "r", encoding="utf-8") as f:
    file_contents = f.read()
    characterName_list = eval(file_contents)
len(characterName_list)

582507

# ARMORIES

In [95]:
for i in range(len(characterName_list)):
    # 이미 입력된 값이면 다음 이름
    result = lok.check_name_already_in(characterName_list[i], cursor)
    if result[0] == 1: continue
    
    # calling API
    try:
        profile_responses = lok.get_total_profile_responses(characterName_list[i])
        if profile_responses == None:
            lok.remove_characterName(characterName_list, i, "character response is None")
            continue   
    except Exception as e:
        print("Error:", e)
        print("CharacterName:", characterName_list[i])
        lok.remove_characterName(characterName_list, i, e)
        continue
        
    try:
        values = lok.get_profile_values(profile_responses)
        lok.insert_raw_character_data(values)
        if i % 2000 == 0:
            print("Record inserted successfully:", i)
            print("Current time:", datetime.datetime.now())
    except Exception as e:
        lok.remove_characterName(characterName_list, i, e)
        continue


## Check data
# sql = """
# SELECT * FROM lostark.raw_character_data_table;
# """
# cursor.execute(sql)
# result = cursor.fetchall()
# pprint(result[:10])

In [69]:
db.close()